In [46]:
import pandas as pd
import numpy as np

In [47]:
# This data represents demand by day for each size of product "v"
data_v = {
    'Chica' : [12, 15, 10],
    'Mediana' : [8, 6, 2],
    'Grande' : [4, 8, 3]
}

# This data represents demand by day for each size of product "r"
data_r = {
    'Chica' : [11, 14, 9],
    'Mediana' : [7, 5, 1],
    'Grande' : [3, 7, 2]
}

T = np.array([25, 30, 45]) / 60  # Cleaning times in hours
U = np.array([60, 0, 30]) / 60  # Cooldown times in hours
rM = np.array([80, 80, 80])  # Production rates in L/hour
C = np.array([100, 40, 50])  # Minimum batch capacities

In [48]:
# We calculate the total liters per day by multiplying the demand per size by the
# vector [0.5, 0.3, 0.2]
def litros_totales(row):
    return np.dot(row, [0.5, 0.3, 0.2])

In [49]:
df_v = pd.DataFrame(data_v)
df_r = pd.DataFrame(data_r)

df_v['L_total'] = df_v.apply(litros_totales, axis=1)
df_r['L_total'] = df_r.apply(litros_totales, axis=1)


# Combine "v" and "r" into a single DataFrame
df_v['Product'] = 'Verde'
df_r['Product'] = 'Roja'

df_v['Day'] = df_v.index + 1
df_r['Day'] = df_r.index + 1

df = pd.concat([df_v, df_r], ignore_index=True)

In [50]:
# Scheduling Backward
schedule = []
current_time = 24  # End of the day (24 hours)
process_id = 1  # Counter for processes
sequence_id = 1  # Sub-process sequence tracker

# Iterate over days in reverse order
for day in sorted(df['Day'].unique(), reverse=True):
    daily_data = df[df['Day'] == day]
    for _, row in daily_data.iterrows():
        total_liters = row['L_total']
        product = row['Product']

        if total_liters > 0:
            # Choose the best machine based on minimum batch size and production rate
            selected_machine = np.argmax(C)
            batch_size = C[selected_machine]
            production_rate = rM[selected_machine]
            cleaning_time = T[selected_machine]
            cooldown_time = U[selected_machine]

            # Calculate batches and production times
            batches = int(total_liters // batch_size)
            remainder = total_liters % batch_size

            # Schedule batches
            for i in range(batches):
                end_time = current_time
                process_time = batch_size / production_rate
                start_time = end_time - process_time
                schedule.append({
                    'Proceso': process_id,
                    'Máquina': selected_machine + 1,
                    'Producto': product,
                    'Cantidad': batch_size,
                    'Secuencia': f"{process_id}.{sequence_id}",
                    'Tiempo de proceso': process_time,
                    'Tiempo entre procesos': 0,  # To be adjusted later if needed
                    'Hora Inicio': start_time,
                    'Hora Fin': end_time
                })
                current_time = start_time - cleaning_time
                sequence_id += 1

            # Schedule the remainder batch if it exists
            if remainder > 0:
                end_time = current_time
                process_time = remainder / production_rate
                start_time = end_time - process_time
                schedule.append({
                    'Proceso': process_id,
                    'Máquina': selected_machine + 1,
                    'Producto': product,
                    'Cantidad': remainder,
                    'Secuencia': f"{process_id}.{sequence_id}",
                    'Tiempo de proceso': process_time,
                    'Tiempo entre procesos': 0,  # To be adjusted later if needed
                    'Hora Inicio': start_time,
                    'Hora Fin': end_time
                })
                current_time = start_time - cleaning_time
                sequence_id += 1

            # Account for cooldown
            current_time -= cooldown_time

        process_id += 1
        sequence_id = 1  # Reset sequence for the next process

# Convert schedule to DataFrame for better visualization
schedule_df = pd.DataFrame(schedule)
schedule_df.sort_values(by=['Proceso', 'Secuencia'], inplace=True)

# Display the final schedule
schedule_df

,Proceso,Máquina,Producto,Cantidad,Secuencia,Tiempo de proceso,Tiempo entre procesos,Hora Inicio,Hora Fin
0,1,1,Verde,6.2,1.1,0.07750,0,23.922500,24.000000
1,2,1,Roja,5.2,2.1,0.06500,0,22.440833,22.505833
2,3,1,Verde,10.9,3.1,0.13625,0,20.887917,21.024167
3,4,1,Roja,9.9,4.1,0.12375,0,19.347500,19.471250
4,5,1,Verde,9.2,5.1,0.11500,0,17.815833,17.930833
5,6,1,Roja,8.2,6.1,0.10250,0,16.296667,16.399167
